## Welcome!

Welcome to the Zero to GPT course!  This course will take you from no deep learning knowledge to being able to train your own GPT model.

At a high level, GPT works by predicting what words should come after your prompt.  Here's an example of some text you can feed into GPT, and a response:

![gpt text](images/intro/gpt_text.svg)

GPT models are types of deep learning models.  Deep learning is a subfield of machine learning, which is a subfield of AI.  In deep learning, we train a neural network composed of many layers to transform an input into an output.

Neural networks can't understand text directly, so GPT has to convert your prompt to numbers, make predictions, then convert the predictions back to text.  The process works like this at a very high level:

![gpt steps](images/intro/gpt_steps.svg)


The key part of GPT is the prediction step, where the network predicts what words will come next.  To do this, the network uses its parameters.  You can think of parameters like memory.  This memory has been trained on a lot of data (gigabytes or terabytes).  Parameters enable neural networks to make accurate predictions about what should come after your prompt.

The smallest GPT models have around 100M parameters, and the largest have 176B.  They use parameters to transform your input to predictions - usually by adding the parameters to your input, or multiplying your input by the parameters.

By the end of this course, you'll understand how to train deep neural networks that work like GPT.  This course is split up into three parts:

1. Understanding deep learning - how to create and train deep neural networks, including gradient descent and backpropagation.
2. Deep learning for NLP - deep learning architectures to work with text, including RNNs, encoder/decoders, attention, and transformers.
3. Scaling up models - putting together the building blocks to train a model with a large number of parameters.

Check out the [README](https://github.com/VikParuchuri/zero_to_gpt/blob/master/README.md) for a full lesson overview.

## Teaching method

This course will get you ready to do meaningful work with deep learning.  We'll balance theory and practical application.  Without theory, you won't understand how to apply deep learning in the real world.  But theory shouldn't overwhelm - it should aid you in actually doing the work.

In each lesson, you'll learn the theory behind what you're doing, and code complete implementations that solve problems.  For example, in the dense neural networks lesson, you'll learn the theory - backpropagation, why nonlinearities are useful, and how multiple layers work.  Then you'll code a complete neural network using NumPy that predicts the weather.

I'll explain concepts in multiple ways - diagrams, text, code, and video.  Deep learning is complicated, and it can take multiple explanations before a concept clicks.

### What this course is not

- This is not a course where you'll just import PyTorch and type code without understanding what you're doing.  We'll build up your knowledge step by step.
- This is not a mathematically rigorous course.  We'll include the math, but we'll focus on building intuition and understanding first.  For a mathematically rigorous resource, I recommend the [deep learning book](https://www.deeplearningbook.org/).

## Who am I?

My name is Vik Paruchuri.  I've won ML competitions, worked as an ML engineer, and started Dataquest, which has taught data skills to 1M+ people.  Before all that, I was a self-taught programmer who dreamed of predicting the stock market.

This course is for anyone dreaming of building their own deep learning models.

## Prerequisites

This course is designed to be accessible, with as few prereqs as possible:

- You'll need to understand Python to take this course, including for loops, functions, and classes.  The first part of [this Dataquest path](https://www.dataquest.io/path/data-scientist/) will teach you what you need.
- Experience with [NumPy](https://www.dataquest.io/course/pandas-fundamentals) and linear algebra are also necessary.  If you don't have any experience, we'll cover the basics in the [next lesson](https://github.com/VikParuchuri/zero_to_gpt/blob/master/explanations/linalg.ipynb).

If you already know NumPy and the basics of linear algebra, feel free to skip to the [next lesson](https://github.com/VikParuchuri/zero_to_gpt/blob/master/explanations/linreg.ipynb).